In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 150

In [222]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

# Data
# from transat.data import HYPOTHETICAL_SUBMISSION_DATE
from transat.data.load import download_historical, load_historical
from transat.data.split import split_historical
from transat.data.transform import preprocess_historical_basic, dataframe_to_array

# Metric
from transat.metric import mae

# Scenario/Simulation
from transat.data.scenario import generate_scenario

In [223]:
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-09-30")

In [224]:
download_historical()
df = load_historical()

In [225]:
df = preprocess_historical_basic(df, norm_by_pop='False')

In [226]:
df

,CountryName,RegionName,GeoID,Date,NewCases,NewCasesSmoothed7Days,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99955,Zimbabwe,NaN,Zimbabwe__nan,2020-12-18,181.0,0.000009,0.25,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,0.25,0.25,0.75
99956,Zimbabwe,NaN,Zimbabwe__nan,2020-12-19,104.0,0.000009,0.25,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,0.25,0.25,0.75
99957,Zimbabwe,NaN,Zimbabwe__nan,2020-12-20,174.0,0.000010,0.25,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,0.25,0.25,0.75
99958,Zimbabwe,NaN,Zimbabwe__nan,2020-12-21,97.0,0.000010,0.25,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,0.25,0.25,0.75


In [227]:
df.to_csv('df_dataframe_preprocessed.csv')

### Add OWID data

In [228]:
xp = pd.read_csv('./data/xprize_full_data.csv')

In [229]:
xp

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Latitude,Longitude,population_density,median_age,aged_65_older,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,Population
0,Afghanistan,AFG,NaN,NaN,NAT_TOTAL,33.939110,67.709953,54.422,18.6,2.5810,...,2.35,597.029,9.59,5.8,30.9,37.746,0.50,64.83,0.498,38928341.0
1,Angola,AGO,NaN,NaN,NAT_TOTAL,-11.202692,17.873887,23.890,16.8,2.4050,...,2.35,276.045,3.94,5.8,30.9,26.664,2.20,61.15,0.581,32866268.0
2,Albania,ALB,NaN,NaN,NAT_TOTAL,41.153332,20.168331,104.871,38.0,13.1880,...,1.10,304.195,10.08,7.1,51.2,47.964,2.89,78.57,0.785,2877800.0
3,Andorra,AND,NaN,NaN,NAT_TOTAL,42.546245,1.601554,163.755,29.2,6.0035,...,2.35,109.135,7.97,29.0,37.8,47.964,2.20,83.73,0.858,77265.0
4,United Arab Emirates,ARE,NaN,NaN,NAT_TOTAL,23.424076,53.847818,112.442,34.0,1.1440,...,2.35,317.840,17.26,1.2,37.4,47.964,1.20,77.97,0.863,9890400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Vanuatu,VUT,NaN,NaN,NAT_TOTAL,-15.376706,166.959158,22.662,23.1,4.3940,...,13.20,546.300,12.02,2.8,34.5,25.209,2.20,70.47,0.603,307150.0
246,Yemen,YEM,NaN,NaN,NAT_TOTAL,15.552727,48.516388,53.508,20.3,2.9220,...,18.80,495.003,5.35,7.6,29.2,49.542,0.70,66.12,0.452,29825968.0
247,South Africa,ZAF,NaN,NaN,NAT_TOTAL,-30.559482,22.937506,46.754,27.3,5.3440,...,18.90,200.380,5.52,8.1,33.2,43.993,2.32,64.13,0.699,59308690.0
248,Zambia,ZMB,NaN,NaN,NAT_TOTAL,-13.133897,27.849332,22.995,17.7,2.4800,...,57.50,234.499,3.94,3.1,24.7,13.938,2.00,63.89,0.588,18383956.0


In [230]:
indexes = xp[xp.CountryName=='Brazil'].index
xp.drop(labels = indexes[0:-1], axis=0, inplace=True)

xp['GeoID'] = xp["CountryName"] + "__" + xp["RegionName"].astype(str)

df_xp = df.merge(xp[['Latitude', 'Longitude', 'population_density',
       'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index', 'Population', 'GeoID']], left_on='GeoID', right_on='GeoID', how='outer')

In [231]:
df_xp

,CountryName,RegionName,GeoID,Date,NewCases,NewCasesSmoothed7Days,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,Population
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.000000,0.00,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.000000,0.00,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.000000,0.00,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.000000,0.00,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.000000,0.00,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84604,Zimbabwe,NaN,Zimbabwe__nan,2020-12-18,181.0,0.000009,0.25,0.5,0.5,0.75,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,14862927.0
84605,Zimbabwe,NaN,Zimbabwe__nan,2020-12-19,104.0,0.000009,0.25,0.5,0.5,0.75,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,14862927.0
84606,Zimbabwe,NaN,Zimbabwe__nan,2020-12-20,174.0,0.000010,0.25,0.5,0.5,0.75,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,14862927.0
84607,Zimbabwe,NaN,Zimbabwe__nan,2020-12-21,97.0,0.000010,0.25,0.5,0.5,0.75,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,14862927.0


In [232]:
print("Spliting at : ", HYPOTHETICAL_SUBMISSION_DATE)
# where ``df_train`` contains all data <= ``split_date`` and ``df_test`` contains all data > ``split_date``
df_train, df_test = split_historical(df_xp, HYPOTHETICAL_SUBMISSION_DATE)
df_train, df_test = split_historical(df_xp, HYPOTHETICAL_SUBMISSION_DATE)

Spliting at :  2020-09-30


In [233]:
nb_lookback_days = 30
nb_lookahead_days = 1
sequence_format = True
neg_npis = True

(X_train, y_train), (X_cols, y_col) = dataframe_to_array(df_train, nb_lookback_days=nb_lookback_days, 
    nb_lookahead_days=nb_lookahead_days, sequence_format=sequence_format, neg_npis=neg_npis)
(X_test, y_test), _ = dataframe_to_array(df_test, nb_lookback_days=nb_lookback_days,
    nb_lookahead_days=nb_lookahead_days, sequence_format=sequence_format, neg_npis=neg_npis)

X_train, y_train = X_train.reshape(X_train.shape[0], -1), y_train.reshape(-1)
X_test, y_test = X_test.reshape(X_test.shape[0], -1), y_test.reshape(-1)

print("X_train shape: ", np.shape(X_train))
print("y_train shape: ", np.shape(y_train))
print()
print("X_test  shape: ", np.shape(X_test))
print("y_test  shape: ", np.shape(y_test))

X_train shape:  (57354, 390)
y_train shape:  (57354,)

X_test  shape:  (12087, 390)
y_test  shape:  (12087,)


In [234]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.

from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=20000,
              positive=True,
              selection='random')

# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=20000, positive=True, precompute=True,
      selection='random')

In [235]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 4.197411524274393e-05
Test MAE: 0.0002154749783647259


In [236]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + X_cols[0])
for d in range(-nb_lookback_days, 1):
    for col_name in X_cols[1:]:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Intercept 3.338743038742837e-05


In [237]:
def simulate_scenario(X_scenario, y_scenario):
    # Simulate scenario

    X_sim = X_scenario.copy()
    X_sim_cases = X_sim[:,:,:1]
    X_sim_npis = X_sim[:,:,1:]
    y_sim = np.zeros(np.shape(y_scenario))

    nb_lookback_days = X_sim.shape[1]

    for d in range(y_sim.shape[1]):

        y = model.predict(X_sim.reshape(1,-1))
        y_sim[0,d,0] = max(y[0], 0)

        # Assuming constant NPIs here
        X_sim_npis = np.concatenate([X_sim_npis[:,1:], X_sim_npis[:,-1:]], axis=1)
        X_sim_cases = np.concatenate([X_sim_cases[:,1:], y.reshape(-1, 1, 1)], axis=1)

        X_sim =  np.concatenate([X_sim_cases, X_sim_npis], axis=-1)
        X_sim = np.array(X_sim)
    
    return y_sim

In [238]:
def viz_scenario(geo_id, X_scenario, y_scenario, y_sim):
    mae_error = mae(y_scenario, y_sim)

    plt.figure()
    plt.title(geo_id)

    plot_input_x = np.arange(X_scenario.shape[1])
    plot_input_y = X_scenario[:,:,:1].reshape(-1)

    plt.plot(plot_input_x, plot_input_y, label="Input Scenario")

    plot_output_x = np.arange(y_scenario.shape[1])+X_scenario.shape[1]
    plot_output_x = np.concatenate([plot_input_x[-1:], plot_output_x])
    plot_output_y = np.concatenate([plot_input_y[-1:], y_scenario.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Scenario")


    plot_output_y = np.concatenate([plot_input_y[-1:], y_sim.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Simulation")

    plt.ylabel("New Cases")
    plt.xlabel("Days")
    ax = plt.gca()
    plt.text(0.3, 0.5, f"$MAE={mae_error:.2f}$", transform=ax.transAxes)
    plt.legend()
    plt.show()

In [14]:
def interactive_scenario(geo_id):
    nb_future_days=30

    X_scenario , y_scenario = generate_scenario(df_train, df_test, geo_id, nb_lookback_days=nb_lookback_days,
        nb_future_days=nb_future_days, sequence_format=sequence_format)

    y_sim = simulate_scenario(X_scenario, y_scenario)

    viz_scenario(geo_id, X_scenario, y_scenario, y_sim)
    
geo_ids = sorted(df.GeoID.unique())

w_geo_id = widgets.Dropdown(
    options=geo_ids,
    value='France__nan',
    description='GeoID:',
    disabled=False
)

interact(interactive_scenario, geo_id=w_geo_id)

interactive(children=(Dropdown(description='GeoID:', index=60, options=('Afghanistan__nan', 'Albania__nan', 'A…

<function __main__.interactive_scenario(geo_id)>